In [39]:
from math import *
import pygame
import random
import sys
import numpy as np
import os

In [40]:
def rotate(vector, theta):
        theta = theta/1000
        Rotation_matrix = np.matrix([[cos(theta), -sin(theta)], [sin(theta), cos(theta)]])
        return(np.squeeze(np.asarray(Rotation_matrix@vector)))

In [41]:
class Car:
    def __init__(self, position_0, angle=0.0, length=4, max_steering=30, max_acceleration=5.0, scaling=1, max_velocity_0=20):
        self.position = position_0
        self.velocity = np.array([0.0, 0.0])
        self.angle = angle
        self.length = length
        self.max_acceleration = max_acceleration
        self.max_steering = max_steering
        self.max_velocity =  max_velocity_0/scaling
        self.brake_deceleration = 10
        self.free_deceleration = 2

        self.acceleration = 0.0
        self.steering = 0.0

    def update(self, dt, scaling):
        # User input
        pressed = pygame.key.get_pressed()

        if pressed[pygame.K_UP]:
            if self.velocity[0] < 0:
                self.acceleration = self.brake_deceleration
            else:
                self.acceleration += (1/scaling) * dt
        elif pressed[pygame.K_DOWN]:
            if self.velocity[0] > 0:
                self.acceleration = -self.brake_deceleration
            else:
                self.acceleration -= (1/scaling) * dt
        elif pressed[pygame.K_SPACE]:
            if abs(self.velocity[0]) > dt * self.brake_deceleration:
                self.acceleration = -copysign(self.brake_deceleration, self.velocity[0])
            else:
                self.acceleration = -self.velocity[0] / dt
        else:
            if abs(self.velocity[0]) > dt * self.free_deceleration:
                self.acceleration = -copysign(self.free_deceleration, self.velocity[0])
            else:
                if dt != 0:
                    self.acceleration = -self.velocity[0] / dt
        self.acceleration = max(-self.max_acceleration, min(self.acceleration, self.max_acceleration))

        if pressed[pygame.K_RIGHT]:
            self.steering -= 30 * dt
        elif pressed[pygame.K_LEFT]:
            self.steering += 30 * dt
        else:
            self.steering = 0
        self.steering = max(-self.max_steering, min(self.steering, self.max_steering))



        self.velocity += np.array([self.acceleration * dt, 0])
        self.velocity[0] = max(-self.max_velocity, min(self.velocity[0], self.max_velocity))

        if self.steering:
            turning_radius = self.length / sin(radians(self.steering))
            angular_velocity = self.velocity[0] / turning_radius
        else:
            angular_velocity = 0

        self.position += rotate(self.velocity, -self.angle) * dt
        self.angle += degrees(angular_velocity) * dt
    
    def draw(self, car_image, ppu, game):
        rotated_car = pygame.transform.rotate(car_image, self.angle)
        rect = rotated_car.get_rect()
        game.screen.blit(rotated_car, self.position * ppu - (rect.width / 2, rect.height / 2))
        return

In [42]:
class Game:
    def __init__(self):
        pygame.init()
        pygame.display.set_caption("Drone pursuit")
        width = 1280
        height = 720
        self.screen = pygame.display.set_mode((width, height))
        self.clock = pygame.time.Clock()
        self.ticks = 60
        self.exit = False

    def run(self):
        current_dir = os.getcwd()
        car_image_path = os.path.join(current_dir, "car.png")
        car_image = pygame.image.load(car_image_path)
        taille = car_image.get_size()
        scaling = 10
        taille = [taille[0]/scaling, taille[1]/scaling]
        car_image = pygame.transform.scale(car_image, taille)
        car = Car(0, 0, scaling=scaling)
        ppu = 32

        while not self.exit:
            dt = self.clock.get_time() / 1000

            # Event queue
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.exit = True

            # Logic
            car.update(dt, scaling)

            # Drawing
            self.screen.fill((0, 0, 0))
            car.draw(car_image, ppu, self)
            
            pygame.display.flip()

            self.clock.tick(self.ticks)
        pygame.quit()

In [43]:
if __name__ == '__main__':
    game = Game()
    game.run()

KeyboardInterrupt: 

In [ ]:
os.getcwd()

'c:\\Users\\antho\\Desktop\\ENSTA\\PIE'